In [51]:
import urllib.request, json, os, sys
import ssl
import numpy as np
import matplotlib.pyplot as pyplt
import librosa, librosa.display

"""
@author: Elías Bjartur Einarsson, haust 2020
með innblæstri og aðstoð frá Karli Birki Flosasyni
"""

## FUGLAHEITI

íslenska = ["Heiðlóa", "Spói", "Hrossagaukur", "Kría", "Álft", "Grágæs", "Hrafn", "Stari",
"Skógarþröstur", "Maríuerla", "Tjaldur", "Sílamáfur", "Svartþröstur"]

latína = ["Pluvialis+apricaria", "Numenius+phaeopus", "Gallinago+gallinago",
"Sterna+paradisaea", "Cygnus+cygnus", "Anser+anser", "Corvus+corax",
"Sturnus+vulgaris", "Turdus+iliacus", "Motacilla+alba",
"Haematopus+ostralegus", "Larus+fuscus",  "Turdus+merula"]

enska = ["European Golden Plover", "Whimbrel", "Common Snipe", "Arctic Tern",
"Whooper Swan", "Greylag Goose", "Northern Raven", "Common Starling",
"Redwing","White Wagtail", "Eurasian Oystercatcher", "Lesser Black-backed Gull",
"Common Blackbird"]

'''
enska = ["European_golden_plover", "Whimbrel", "Common_Snipe", "Arctic_tern",
"Whooper_swan", "greylag_goose", "northern_raven", "common_starling",
"redwing","white_wagtail", "Eurasian_oystercatcher", "Lesser_Black-backed_Gull",
"Common_Blackbird"]
'''

# Bæta mögulega við
islBæta = ["Þúfutittlingur", "Snjótittlingur", "Stokkönd", "Rjúpa"]
latBæta = ["Anthus+pratensis", "Plectrophenax+nivalis", "Anas+platyrhynchos", "Lagopus+muta"]

tegundir = latína

# Þýðirngar orðabók
latIsl = dict(zip(latína+latBæta, íslenska+islBæta))


# Staðsetning gagna
base_dir = os.path.join('/Volumes', 'Sægátt', 'Birds')

In [11]:
# Athugum fjölda á upptökum sem í boði eru fyrir hverja fuglategund
for tegund in latBæta+tegundir:
    server = "https://www.xeno-canto.org/api/2/recordings?query="
    query = server + tegund

    with urllib.request.urlopen(query) as url: 
        data = json.loads(url.read().decode())
        #Prints the number of recordings for specie  
        print("Fjöldi hljóðupptaka fyrir " + latIsl[tegund] + " : %d " % int(data['numRecordings']))

Fjöldi hljóðupptaka fyrir Þúfutittlingur : 687 
Fjöldi hljóðupptaka fyrir Snjótittlingur : 218 
Fjöldi hljóðupptaka fyrir Stokkönd : 631 
Fjöldi hljóðupptaka fyrir Rjúpa : 59 
Fjöldi hljóðupptaka fyrir Heiðlóa : 403 
Fjöldi hljóðupptaka fyrir Spói : 563 
Fjöldi hljóðupptaka fyrir Hrossagaukur : 634 
Fjöldi hljóðupptaka fyrir Kría : 196 
Fjöldi hljóðupptaka fyrir Álft : 358 
Fjöldi hljóðupptaka fyrir Grágæs : 527 
Fjöldi hljóðupptaka fyrir Hrafn : 1370 
Fjöldi hljóðupptaka fyrir Stari : 1051 
Fjöldi hljóðupptaka fyrir Skógarþröstur : 1179 
Fjöldi hljóðupptaka fyrir Maríuerla : 981 
Fjöldi hljóðupptaka fyrir Tjaldur : 614 
Fjöldi hljóðupptaka fyrir Sílamáfur : 152 
Fjöldi hljóðupptaka fyrir Svartþröstur : 4539 


In [23]:
# Hlaðar niður hljóðskrám

Gogn = {}

verbose_rate = 50
overwriteAudio = False

for tegund in tegundir: 
    print("Næ í hljóðskrár fyrir " + latIsl[tegund])
    server = "https://www.xeno-canto.org/api/2/recordings?query="
    query = server + tegund
    
    recordings = [] 
  
    with urllib.request.urlopen(query) as url:
        data = json.loads(url.read().decode())
        recordings = data['recordings']
        numRecordings = data['numRecordings']
        # Ath, hér þarf að athuga data['numPages'] ef 
        # stefnt er á að hala meira en 500 skrám per tegund

        
    print("Heildarfjöldi tiltækra hljóðskráa: " + numRecordings)
    print("Hölum niður : " + str(len(recordings)))
    eventualDls = len(recordings) 
    exception_skipped = []

    # hölum niður hljóðskrám
    skipped_overwrite = 0
    block_size = 8192 
    directory = os.path.join(base_dir, tegund)
    if not os.path.isdir(directory):
        os.mkdir(directory)
    for rec in range(len(recordings)):
        if(rec % verbose_rate == 0):
            print("Hala niður skrá nr.: " + str(rec))
        try:
            skrá = urllib.request.urlopen("http:" + recordings[rec]['file'])  
        except:
            print('An exception caught')
            exception_skipped.append(rec)
            eventualDls -= 1

        file_name = os.path.join(directory, recordings[rec]['id'] + ".mp3")
        recordings[rec]['path'] = file_name
        
        
        if not os.path.isfile(file_name) or overwriteAudio:
            f = open(file_name, 'wb') 
            while True:
                buff = skrá.read(block_size)
                if not buff:
                    break
                f.write(buff)
            f.close()
        else:
            skipped_overwrite += 1
            if(skipped_overwrite % verbose_rate == 0):
                print("Sleppti að skrifa yfir " + str(skipped_overwrite))
        
    print("Búinn að hala niður fyrir " + latIsl[tegund])   
    print('Í heildina ' + str(eventualDls))
    for r in exception_skipped:
        del recording[r]
    Gogn.update({ tegund : recordings[:eventualDls] }) 

print("Búið að hlaða niður öllum skrám")

# Vistum upplýsingar um gagnasafnið
dict = Gogn
jsonDat = json.dumps(dict)
with open("Gogn" + ".json","w") as f:
    f.write(jsonDat)
del dict, jsonDat

Næ í hljóðskrár fyrir Heiðlóa
Heildarfjöldi tiltækra hljóðskráa: 414
Hölum niður : 414
Hala niður skrá nr.: 0
Hala niður skrá nr.: 50
Sleppti að skrifa yfir 50
Hala niður skrá nr.: 100
Sleppti að skrifa yfir 100
Hala niður skrá nr.: 150
Sleppti að skrifa yfir 150
Hala niður skrá nr.: 200
Sleppti að skrifa yfir 200
Hala niður skrá nr.: 250
Sleppti að skrifa yfir 250
Hala niður skrá nr.: 300
Sleppti að skrifa yfir 300
Hala niður skrá nr.: 350
Sleppti að skrifa yfir 350
Hala niður skrá nr.: 400
Sleppti að skrifa yfir 400
Búinn að hala niður fyrir Heiðlóa
Í heildina 414
Næ í hljóðskrár fyrir Spói
Heildarfjöldi tiltækra hljóðskráa: 566
Hölum niður : 500
Hala niður skrá nr.: 0
Hala niður skrá nr.: 50
Sleppti að skrifa yfir 50
Hala niður skrá nr.: 100
Sleppti að skrifa yfir 100
Hala niður skrá nr.: 150
Sleppti að skrifa yfir 150
Hala niður skrá nr.: 200
Sleppti að skrifa yfir 200
Hala niður skrá nr.: 250
Sleppti að skrifa yfir 250
Hala niður skrá nr.: 300
Sleppti að skrifa yfir 300
Hala niður

In [ ]:
# óunnin hljóðrit
import audioread
import soundfile
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

overwriteSpectrograms = False

with open("Gogn" + ".json") as f:
    Gögn = json.loads(f.read()) 

for tegund in tegundir:
    print('Undirbý hljóðgögn fyrir ' + latIsl[tegund])
    path_dir = os.path.join(base_dir, 'Spectrograms', tegund)
    if not os.path.isdir(path_dir):
        os.mkdir(path_dir)
    for i in range(len(Gögn[tegund])):
        if i%verbose_rate == 0:
            print('Meðhöndla upptöku nr. ' + str(i))
        file_dir = os.path.join(path_dir, str(Gögn[tegund][i]['id']))
        if not os.path.isfile(file_dir) or overwriteSpectrograms: 
            path = Gögn[tegund][i]['path']
            audio, _ = librosa.load(path)
            audio, _ = librosa.effects.trim(audio)
            melSp = librosa.feature.melspectrogram(audio, n_mels=128, hop_length=512, n_fft=2048)
            DbMel = librosa.amplitude_to_db(melSp, ref=np.max)
            Gögn[tegund][i]['recLen'] = DbMel.shape[1]
            np.save(file_dir, DbMel)
            # ath. að hér gæti ég haft normalizeringu, sem stendur er hún í datagenerator

    print('Búinn með ' + latIsl[tegund])
print('Allt klárað!')


In [21]:
def cutLines(mel, numCut):
    mel[:numCut] = np.zeros([mel[:numCut].shape[0], mel.shape[1]], dtype=np.float32)
    return mel

In [23]:
# author: Karl Birkir Flosason

def norma(S_norm):
    _max = np.max(S_norm)
    _min = np.min(S_norm)
    S_norm = (S_norm-_min)/(_max-_min)
    return S_norm

def silenceCutterTwoDim(S_in, thresholdFactor=1.1, skipZone=4, freqWidth=5):
    """
    auth: KarlBF
    Finds the line (freqency vector) with the highest variance. 
    Selects those columns (time vectors) where the mean of the frequency vector
    is above the wider mean by thresholdFactor. Wider mean is over max var line,
    +/- freqwidth.
    thresholdfactor 1 is the mean over the frequency band
    skipZone is a margin that also gets selected. 
    LastKeepPos prevents double-selecting columns which could lead to returning
    a longer output than input
    """
    # Find the freq band with the highest difference between min and max
    _minMax, _minMaxLine, _minMaxDelta = 0, 0, 0;
    for i in range(S_in.shape[0]):
        _minMaxDelta = np.max(S_in[i])-np.min(S_in[i])
        if(_minMaxDelta >= _minMax):
            _minMax = _minMaxDelta
            _minMaxLine = i
    S_t = np.zeros([S_in.shape[0], 1], dtype=np.float32)
    skipFlag = False    # True if appended last, false if skipped last
    lastKeepPos=0   
    # Establish upper and lower freq bounds 
    lowerBound, higherBound = _minMaxLine-freqWidth, _minMaxLine+freqWidth
    assert(lowerBound >= 0 & higherBound <= S_t.shape[0])
    threshold = np.mean( S_in[lowerBound:higherBound] ) * thresholdFactor   
    # iterate by columns, appending if higher than threshold
    for i in range(S_in.shape[1]):
        if np.max(S_in[lowerBound:higherBound, i]) > threshold:
            if not skipFlag:
                if (i-skipZone) < lastKeepPos:
                    S_t = np.append(S_t, S_in[:, lastKeepPos:i-1], axis=1)
                    lastKeepPos=i
                else:
                    S_t = np.append(S_t, S_in[:, i-skipZone:i-1], axis=1)
                    lastKeepPos=i
            skipFlag = True
            S_t = np.append(S_t, S_in[:,i:i+1], axis=1)
            lastKeepPos=i
        if np.max(S_in[lowerBound:higherBound, i]) < threshold:
            if skipFlag:  # True ef nybuid ad vista 
                S_t = np.append(S_t, S_in[:,i:i+skipZone], axis=1)
                i = i+skipZone
                lastKeepPos=i
                skipFlag = False
    return S_t


In [16]:
def process(path): 
    audio, _ = librosa.load(path)
    audio, _ = librosa.effects.trim(audio)
    melSp = librosa.feature.melspectrogram(audio, n_mels=128, hop_length=512, n_fft=2048)
    melSp = cutLines(melSp, 10)
    melSp = silenceCutterTwoDim(melSp)
    melSp = norma(melSp) 
    DbMel = librosa.amplitude_to_db(melSp, ref=np.max)
    return DbMel

In [ ]:
# unnin hljóðrit
import audioread
import soundfile
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

overwriteProcessedSpectrograms = False

with open("Gogn" + ".json") as f:
    Gögn = json.loads(f.read()) 

for tegund in tegundir:
    print('Undirbý hljóðgögn fyrir ' + latIsl[tegund])
    path_dir = os.path.join(base_dir, 'Processed_Spectrograms', tegund)
    if not os.path.isdir(path_dir):
        os.mkdir(path_dir)
    for i in range(len(Gögn[tegund])):
        if i%50 == 0:
            print('Meðhöndla upptöku nr. ' + str(i))
        file_dir = os.path.join(path_dir, str(Gögn[tegund][i]['id']))
        if not os.path.isfile(file_dir) or overwriteProcessedSpectrograms: 
            path = Gögn[tegund][i]['path']
            DbMel = process(path)
            lengd = DbMel.shape[1]
            Gögn[tegund][i]['shortRecLen'] = lengd
            Gögn[tegund][i]['procMelPath'] = file_dir + '.npy'
            np.save(file_dir, DbMel)
            # athuga normalizeringu, núna gerð fyrir Db og í datagenerator

    print('Búin með ' + latIsl[tegund])

dict = Gögn
jsonDat = json.dumps(dict)
with open("Gogn" + ".json", 'w') as f:
    f.write(jsonDat)
del dict, jsonDat

print('Búin')    